<a href="https://colab.research.google.com/github/Nischal2015/ncit-workshop/blob/main/3_rag/1_simple_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 1
### Building a "Traditional" Rag System

### Run this if you are using Google Colab

In [ ]:
# !pip install langchain langchain-openai langchain-qdrant

In [ ]:
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
# os.environ["LANGSMITH_PROJECT"] = "ncit-workshop"
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["QDRANT_API_KEY"] = userdata.get("QDRANT_API_KEY")
# os.environ["QDRANT_URL"] = "qdrant-host"

### Run this if you are running VSCode

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from core import load_vault_env

load_vault_env()

### Imports

In [ ]:
import os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel

### Initialization

#### Credentials

In [22]:
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

#### Initialize clients

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.2)
vector_store = QdrantVectorStore.from_existing_collection(
    collection_name="ncit-workshop-simple-rag",
    embedding=embeddings,
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
)

### The Actual RAG (Retrieval Augmented Generation)

#### Retriever Function

In [ ]:
from qdrant_client import models


def retrieve_relevant_docs(query: dict[str, str], k: int = 3):
    question = query["question"]
    category = query.get("filter", None)

    print(f"\n[CHAIN LOG] Searching for: '{query} in '{category or 'ALL'}'")

    q_filter = None
    if category:
        q_filter = models.Filter(
            must=[
                models.FieldCondition(
                    key="metadata.category", match=models.MatchValue(value=category)
                )
            ]
        )

    # Perform search with scores
    results = vector_store.similarity_search_with_score(
        query=question, k=k, filter=q_filter
    )

    # Filter by Threshold & Format
    valid_context = []
    for doc, score in results:
        if score >= 0.5:
            valid_context.append(
                f"Policy ID: {doc.metadata['policy_id']}\n"
                f"Topic: {doc.metadata['topic']}\n"
                f"Rule: {doc.page_content}"
            )

    if not valid_context:
        return "NO RELEVANT DOCUMENT FOUND."

    return "\n\n".join(valid_context)

#### Prompt

In [ ]:
template = """You are a strictly factual HR Policy Bot.
Answer the question based ONLY on the context provided below. 
Cite the Policy ID and topic for every fact you state.

Context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

#### Build the chain

In [ ]:
# We use RunnableParallel to pass the question through, while calculating context
rag_chain = (
    RunnableParallel(
        {
            "context": RunnableLambda(retrieve_relevant_docs),
            "question": lambda x: x["question"],
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain

### Execution Scenarios

##### Scenario 1: Searching with a filter (The "Happy Path")

In [ ]:
# We explicitly tell it to look in 'finance' because the user is asking about money.
response = rag_chain.invoke(
    {
        "question": "What is the hotel spending limit for major metro areas like NYC?",
        "filter": "finance",
    }
)
print(f"\n🤖 [AI REPLY]:\n{response}")

##### Scenario 2: Searching with a filter but asking an irrelevant question

In [ ]:
# This shows why filtering is safer. It won't accidentally find IT password rules
# just because they contain the word "secure" or "access".
response = rag_chain.invoke(
    {"question": "How many characters long must passwords be?", "filter": "finance"}
)
print(f"\n🤖 [AI REPLY]:\n{response}")
# Expect: No results or low scores because we forced it to look in Finance.

##### Scenario 3: General Search (No filter)

In [ ]:
response = rag_chain.invoke({"question": "What is the policy on VPN access?"})
print(f"\n🤖 [AI REPLY]:\n{response}")